<img width="8%" alt="Hugging Face.png" src="https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/.github/assets/logos/Hugging%20Face.png" style="border-radius: 15%">

# Hugging Face - Few Shot Learning with Inference API
<a href="https://bit.ly/3JyWIk6">Give Feedback</a> | <a href="https://github.com/jupyter-naas/awesome-notebooks/issues/new?assignees=&labels=bug&template=bug_report.md&title=Hugging+Face+-+Few+Shot+Learning+with+Inference+API:+Error+short+description">Bug report</a>

**Tags:** #huggingface #ml #few_shot_learning #prompt #inference_api #ai #text

**Author:** [Saurabh Arjun Sawant](https://www.linkedin.com/in/srsawant34/)

**Last update:** 2023-11-08 (Created: 2023-11-08)

**Description:** This notebook demonstrates how to utilize the <a href="https://huggingface.co/docs/inference-endpoints/index">inference endpoints</a> (additional information can be found here: <a href="https://huggingface.co/docs/api-inference/index#-hosted-inference-api">link</a>) of hugging face models. Additionally, it demonstrates how to use few shot learning for a specific task in a model.

## Input

### Install Packages

In [ ]:
!pip install -q datasets

### Import Libraries


In [ ]:
from datasets import load_dataset
import numpy as np
import requests
import json
import naas

### Add the Model and API token

#### Steps to get API token
- Create an account on [Hugging Face](https://huggingface.co)
- Log in, and click on profile icon (top right corner)
- Go to settings
- Click on [Access tokens](https://huggingface.co/settings/tokens)
- Now, create a new access token with name: `GPT_INFERENCE` and role: `read`
- Copy the generated token and paste it below

We use GPT based models since they excel in few-shot learning due to their ability to generate coherent and contextually relevant responses based on limited examples, capturing relationships in data more effectively than many other large language models.
In this demonstration, we will utilize the <a href="https://huggingface.co/EleutherAI/gpt-neo-1.3B">gpt-neo-1.3B</a> model; additional GPT-based models can be explored <a href="https://huggingface.co/EleutherAI">here</a>. Developed by <a href="https://www.eleuther.ai/">EleutherAI</a>, GPT⁠-⁠Neo is a series of transformer-based language models built on the GPT architecture. EleutherAI aims to create a model of GPT⁠-⁠3's scale and provide open access.

In [ ]:
# Uncomment to store the environment variable in Naas cloud
# naas.secret.add("GPT_INFERENCE", "Paste_token_here")

# Set the environment variables
MODEL = "EleutherAI/gpt-neo-1.3B"
API_TOKEN = naas.secret.get("GPT_INFERENCE")

## Model

### Define function to make API calls to Hugging Face endpoints

In [ ]:
def query(
        payload='', 
        model = MODEL,
        parameters = {
            'max_new_tokens':5,
            'temperature': 0.5
        }, 
        options = {
            'use_cache': False
        }
    ):
    API_URL = f"https://api-inference.huggingface.co/models/{model}"
    headers = {"Authorization": f"Bearer {API_TOKEN}"}
    body = {"inputs":payload,'parameters':parameters,'options':options}
    
    try:
        response = requests.request("POST", API_URL, headers=headers, data= json.dumps(body))
        return response.json()[0]['generated_text']
    except:
        return "Error: " + " ".join(response.json()['error'])

## Output

><b>The model usually takes time to load in the hugging face server. For example, model gpt-neo-1.3B takes approximately 212 seconds</b>

### Zero-shot

Zero-shot learning means to generate meaningful responses from model for tasks or topics it has never been explicitly trained on, showcasing a capacity to generalize and understand novel concepts without specific examples during training.

In [ ]:
prompt = """
Sentence: I loved todays movie.
Sentiment: """

response = query(payload=prompt, model=MODEL)
print(response)

### One-shot

One-shot learning refers to the model's ability to understand and generate meaningful responses after being exposed to a single example or prompt during the inference phase, showcasing its capacity to generalize knowledge from limited input.

In [ ]:
prompt = """
Sentence: I loved todays movie.
Sentiment: positive

#####

Sentence: I didn't like the action.
Sentiment: """

response = query(payload=prompt, model=MODEL)
print(response)

### Two-shot

Similar to one-shot, we will have the model exposed to two examples to generalize knowledge and make predictions.

In [ ]:
prompt = """
Sentence: I loved todays movie.
Sentiment: positive

#####

Sentence: I didn't like the action.
Sentiment: negative

#####

Sentence: Liked the direction and scene settings.
Sentiment: """

response = query(payload=prompt, model=MODEL)
print(response)

### Few-shot learning with custom dataset

You can also use any custom dataset and generate prompts like above. For example, below we will use <a href="https://huggingface.co/datasets/carblacac/twitter-sentiment-analysis">twitter-sentiment-analysis</a>. More datasets in huggingface can be found <a href="https://huggingface.co/datasets">here</a>.

In [ ]:
def generate_prompt_with_examples(data, target_col, num_of_examples = 0):
    examples = np.random.choice(data, num_of_examples + 1)
    prompts = []
    for example in examples:
        review = example["text"]
        sentiment = "positive" if example[target_col] else "negative"
        prompt = f"Sentence: {review}\nSentiment: {sentiment}\n"
        prompts.append(prompt)
    return """\n#####\n\n""".join(prompts)[:-9]

data = load_dataset('carblacac/twitter-sentiment-analysis')

In [ ]:
prompt = generate_prompt_with_examples(data=data['train'], target_col="feeling", num_of_examples=2)
print(prompt)

In [ ]:
response = query(payload=prompt, model=MODEL)
print(response)